# Đề bài: Sử dụng mạng học sâu để phân loại hình ảnh đất đai từ ảnh vệ tinh
## Model: SegNet
## Data: EuroSat-RGB + BigEarthNet


In [ ]:
!rm -rf /kaggle/working/


# 1. Clone repo


In [ ]:
# Clone từ nhánh khác (thay 'branch-name' bằng tên nhánh bạn muốn)
# Ví dụ: nhánh 'segnet', 'dev', 'feature/segnet', v.v.
BRANCH_NAME = "segnet"  # Thay đổi tên nhánh ở đây

# Cách 1: Clone trực tiếp từ branch (nếu branch tồn tại)
import os
os.system(f'git clone -b {BRANCH_NAME} https://github.com/yuh-tech/ADCV_2025 /kaggle/working//Final_exam')

# Cách 2: Nếu cách 1 không hoạt động, clone từ main rồi checkout sang branch khác:
# !git clone https://github.com/yuh-tech/ADCV_2025 /kaggle/working//Final_exam
# %cd /kaggle/working//Final_exam
# !git checkout {BRANCH_NAME}


# 2. Cài đặt dependencies


In [ ]:
!pip install -r /kaggle/working/Final_exam/requirements.txt


In [ ]:
!python /kaggle/working/Final_exam/setup_kaggle.py


# 3. Setup và Import


In [ ]:
# ============================================================================
# SETUP AND IMPORTS
# ============================================================================

import sys
import os
from pathlib import Path
import matplotlib.pyplot as plt
import numpy as np
import torch
import pandas as pd

# Auto-detect environment
IS_KAGGLE = os.path.exists('/kaggle/input')

if IS_KAGGLE:
    print("Running on Kaggle environment")
    # On Kaggle, repo is cloned to /kaggle/working/Final_exam
    project_root = Path('/kaggle/working/Final_exam')
else:
    print("Running on local environment")
    # On local, notebook is in notebooks/, so go up one level
    project_root = Path.cwd().parent

# Add project root to path
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print(f"Python path updated")


In [ ]:
# Import configurations and modules
from config import (
    EUROSAT_PATH, METADATA_PATH, BIGEARTHNET_FOLDERS, 
    REFERENCE_MAPS_FOLDER, CORINE_TO_EUROSAT, CLASS_NAMES, NUM_CLASSES,
    IS_KAGGLE
)
from src.data import (
    EuroSATDataset, BigEarthNetSegmentationDataset,
    get_classification_train_augmentation, get_val_augmentation,
    get_segmentation_train_augmentation
)
# Import SegNet models từ src/models/segnet.py
# Đây là file chứa SegNet architecture được sử dụng trong training
from src.models.segnet import SegNet, SegNetWithPretrainedEncoder
from src.utils.visualization import mask_to_rgb, denormalize_image, COLOR_PALETTE

print("Imports successful!")
print(f"\n Paths configuration:")
print(f"  EuroSAT: {EUROSAT_PATH}")
print(f"  Exists: {EUROSAT_PATH.exists()}")
print(f"\n  Metadata: {METADATA_PATH}")
print(f"  Exists: {METADATA_PATH.exists()}")
print(f"\n  Reference Maps: {REFERENCE_MAPS_FOLDER}")
print(f"  Exists: {REFERENCE_MAPS_FOLDER.exists()}")
print(f"\n  BigEarthNet folders: {len(BIGEARTHNET_FOLDERS)} found")
for folder in BIGEARTHNET_FOLDERS:
    print(f"    - {folder} (exists: {folder.exists()})")


## 4. Test SegNet Model Architecture


In [ ]:
# Test SegNet model creation
print("="*70)
print("TESTING SEGNET MODEL")
print("="*70)

# Test standard SegNet
print("\n1. Standard SegNet (without pretrained encoder):")
model_standard = SegNet(
    in_channels=3,
    num_classes=NUM_CLASSES,
    base_channels=64
)

# Count parameters
total_params = sum(p.numel() for p in model_standard.parameters())
print(f"  Total parameters: {total_params:,}")

# Test forward pass
test_input = torch.randn(2, 3, 120, 120)
with torch.no_grad():
    output = model_standard(test_input)
print(f"  Input shape: {test_input.shape}")
print(f"  Output shape: {output.shape}")
print(f"  ✓ Standard SegNet works!")

# Test SegNet with pretrained encoder
print("\n2. SegNet with Pretrained Encoder:")
model_pretrained = SegNetWithPretrainedEncoder(
    encoder_name='resnet50',
    num_classes=NUM_CLASSES,
    encoder_pretrained=True,
    freeze_encoder=False
)

total_params_pretrained = sum(p.numel() for p in model_pretrained.parameters())
print(f"  Total parameters: {total_params_pretrained:,}")

with torch.no_grad():
    output_pretrained = model_pretrained(test_input)
print(f"  Input shape: {test_input.shape}")
print(f"  Output shape: {output_pretrained.shape}")
print(f"  ✓ SegNet with pretrained encoder works!")

print("\n" + "="*70)
print("✅ SegNet model architecture verified!")
print("="*70)


## 5. Test Data Loading


In [ ]:
print("="*70)
print("TESTING EUROSAT DATASET")
print("="*70)

# Create transforms
train_transform = get_classification_train_augmentation(64, strength='light')
val_transform = get_val_augmentation(64)

# Load datasets
try:
    train_dataset = EuroSATDataset(EUROSAT_PATH, 'train', train_transform)
    val_dataset = EuroSATDataset(EUROSAT_PATH, 'val', val_transform)
    test_dataset = EuroSATDataset(EUROSAT_PATH, 'test', val_transform)
    
    print(f"✓ Train samples: {len(train_dataset):,}")
    print(f"✓ Val samples: {len(val_dataset):,}")
    print(f"✓ Test samples: {len(test_dataset):,}")
    print(f"✓ Classes: {train_dataset.classes}")
    
    # Get class distribution
    print("\n Class distribution:")
    train_dataset.get_class_distribution()
    
except Exception as e:
    print(f" Error loading EuroSAT: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
print("="*70)
print("LOADING BIGEARTHNET METADATA")
print("="*70)

try:
    metadata_df = pd.read_parquet(METADATA_PATH)
    print(f" Loaded metadata: {len(metadata_df):,} patches")
    print(f"  Columns: {list(metadata_df.columns)}")
    
    # Show split distribution
    print(f"\n Split distribution:")
    print(metadata_df['split'].value_counts())
    
    # For testing, use a subset
    TEST_SUBSET_SIZE = 100  # Adjust based on your needs
    
    train_df = metadata_df[metadata_df['split'] == 'train'].head(TEST_SUBSET_SIZE)
    val_df = metadata_df[metadata_df['split'] == 'validation'].head(50)
    
    print(f"\n Using {len(train_df)} train samples (subset for testing)")
    print(f" Using {len(val_df)} val samples (subset for testing)")
    
except Exception as e:
    print(f" Error loading metadata: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
print("="*70)
print("CREATING BIGEARTHNET DATASET")
print("="*70)

# Create transforms
seg_train_transform = get_segmentation_train_augmentation(120, strength='light')
seg_val_transform = get_val_augmentation(120)

try:
    train_dataset_ben = BigEarthNetSegmentationDataset(
        metadata_df=train_df,
        data_folders=BIGEARTHNET_FOLDERS,
        reference_maps_folder=REFERENCE_MAPS_FOLDER,
        corine_to_eurosat_mapping=CORINE_TO_EUROSAT,
        transform=seg_train_transform,
        num_classes=NUM_CLASSES,
        validate_data=True
    )
    
    val_dataset_ben = BigEarthNetSegmentationDataset(
        metadata_df=val_df,
        data_folders=BIGEARTHNET_FOLDERS,
        reference_maps_folder=REFERENCE_MAPS_FOLDER,
        corine_to_eurosat_mapping=CORINE_TO_EUROSAT,
        transform=seg_val_transform,
        num_classes=NUM_CLASSES,
        validate_data=True
    )
    
    print(f" Created BigEarthNet train dataset: {len(train_dataset_ben)} samples")
    print(f" Created BigEarthNet val dataset: {len(val_dataset_ben)} samples")
    
except Exception as e:
    print(f" Error creating BigEarthNet dataset: {e}")
    import traceback
    traceback.print_exc()


In [ ]:
from torch.utils.data import DataLoader

print("="*70)
print("TESTING DATALOADER WITH SEGNET")
print("="*70)

# Create DataLoader
train_loader = DataLoader(
    train_dataset_ben,
    batch_size=4,
    shuffle=True,
    num_workers=0,  # Use 0 for debugging
    pin_memory=False
)

# Get one batch
try:
    batch = next(iter(train_loader))
    
    print(f" Batch loaded successfully!")
    print(f"  Images: {batch['image'].shape}")
    print(f"  Masks: {batch['mask'].shape}")
    print(f"  Patch IDs: {batch['patch_id']}")
    
    # Test SegNet forward pass with real data
    print(f"\n Testing SegNet forward pass...")
    model_test = SegNetWithPretrainedEncoder(
        encoder_name='resnet50',
        num_classes=NUM_CLASSES,
        encoder_pretrained=True
    )
    model_test.eval()
    
    images = batch['image']
    with torch.no_grad():
        predictions = model_test(images)
    
    print(f"  Input images: {images.shape}")
    print(f"  Predictions: {predictions.shape}")
    print(f"  Masks: {batch['mask'].shape}")
    print(f"   SegNet forward pass successful!")
    
except Exception as e:
    print(f" Error: {e}")
    import traceback
    traceback.print_exc()


## 6. Visualize SegNet Predictions


In [ ]:
# Visualize SegNet predictions
print("="*70)
print("VISUALIZING SEGNET PREDICTIONS")
print("="*70)

import matplotlib.pyplot as plt
from src.utils.visualization import visualize_segmentation

# Get a sample batch
sample_batch = next(iter(train_loader))
images = sample_batch['image']
masks = sample_batch['mask']

# Create model and predict
model_vis = SegNetWithPretrainedEncoder(
    encoder_name='resnet50',
    num_classes=NUM_CLASSES,
    encoder_pretrained=True
)
model_vis.eval()

with torch.no_grad():
    predictions = model_vis(images)

# Visualize
fig, axes = plt.subplots(3, 3, figsize=(15, 15))
fig.suptitle('SegNet Predictions (Untrained Model)', fontsize=16, fontweight='bold')

for i in range(min(3, len(images))):
    # Original image
    img_np = denormalize_image(images[i])
    axes[i, 0].imshow(img_np)
    axes[i, 0].set_title('Original Image', fontweight='bold')
    axes[i, 0].axis('off')
    
    # Ground truth mask
    mask_np = mask_to_rgb(masks[i].numpy(), COLOR_PALETTE)
    axes[i, 1].imshow(mask_np)
    axes[i, 1].set_title('Ground Truth', fontweight='bold')
    axes[i, 1].axis('off')
    
    # Prediction
    pred_mask = torch.argmax(predictions[i], dim=0).cpu().numpy()
    pred_mask_rgb = mask_to_rgb(pred_mask, COLOR_PALETTE)
    axes[i, 2].imshow(pred_mask_rgb)
    axes[i, 2].set_title('SegNet Prediction', fontweight='bold')
    axes[i, 2].axis('off')

plt.tight_layout()
plt.show()

print(" Visualization complete!")


## 7. Training Pipeline

### 7.1. Stage 1: Pre-train Encoder với EuroSAT

Sử dụng script `train_stage1.py` để train encoder với EuroSAT dataset.


In [ ]:
# Chạy Stage 1 training sử dụng script train_stage1.py
# Bạn có thể điều chỉnh các tham số: --epochs, --batch-size, --lr, --model

print("="*70)
print("STAGE 1: ENCODER PRE-TRAINING WITH EUROSAT")
print("="*70)
print("Sử dụng script: train_stage1.py")
print("\nCác tham số có thể điều chỉnh:")
print("  --epochs: Số epochs (mặc định: 50)")
print("  --batch-size: Batch size (mặc định: 64)")
print("  --lr: Learning rate (mặc định: 0.001)")
print("  --model: Model name (mặc định: resnet50)")

# Chạy training
!cd {project_root} && python train_stage1.py --epochs 50 --batch-size 64


In [ ]:
# Kiểm tra kết quả Stage 1
from config import CHECKPOINTS_DIR, VISUALIZATIONS_DIR

stage1_checkpoint_dir = CHECKPOINTS_DIR / 'stage1'
best_model_stage1 = stage1_checkpoint_dir / 'best_model.pth'
encoder_weights = stage1_checkpoint_dir / 'encoder_pretrained.pth'

print("\n" + "="*70)
print("Kiểm tra kết quả Stage 1...")
print("="*70)

if best_model_stage1.exists():
    print(f" Best model saved at: {best_model_stage1}")
else:
    print(f" Best model not found at: {best_model_stage1}")

if encoder_weights.exists():
    print(f" Encoder weights saved at: {encoder_weights}")
else:
    print(f" Encoder weights not found at: {encoder_weights}")

# Load và hiển thị kết quả từ log file
log_file = project_root / 'outputs' / 'logs' / 'stage1_training.log'
if log_file.exists():
    print(f"\n Training log saved at: {log_file}")
    # Đọc và hiển thị một phần log
    with open(log_file, 'r') as f:
        lines = f.readlines()
        print("\n Last few lines from training log:")
        for line in lines[-10:]:
            print(f"  {line.strip()}")
else:
    print(f"\n Training log not found at: {log_file}")


In [ ]:
# Xem thêm thông tin từ checkpoint Stage 1
if best_model_stage1.exists():
    checkpoint = torch.load(best_model_stage1, map_location='cpu')
    print("\n Stage 1 Checkpoint Info:")
    if 'epoch' in checkpoint:
        print(f"  Best epoch: {checkpoint['epoch']}")
    if 'val_accuracy' in checkpoint:
        print(f"  Best validation accuracy: {checkpoint['val_accuracy']:.4f}")
    if 'val_loss' in checkpoint:
        print(f"  Best validation loss: {checkpoint['val_loss']:.4f}")
    print(f"  Checkpoint keys: {list(checkpoint.keys())}")


In [ ]:
# Load và visualize kết quả Stage 1
print("\n" + "="*70)
print("Loading Stage 1 Results for Visualization...")
print("="*70)

# Kiểm tra các file visualization
cm_path_stage1 = VISUALIZATIONS_DIR / 'stage1_confusion_matrix.png'
curves_path_stage1 = VISUALIZATIONS_DIR / 'stage1_training_curves.png'

if cm_path_stage1.exists():
    print(f"✓ Confusion matrix found: {cm_path_stage1}")
    img = plt.imread(cm_path_stage1)
    plt.figure(figsize=(12, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Stage 1: Confusion Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print(f" Confusion matrix not found: {cm_path_stage1}")

if curves_path_stage1.exists():
    print(f"✓ Training curves found: {curves_path_stage1}")
    img = plt.imread(curves_path_stage1)
    plt.figure(figsize=(15, 5))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Stage 1: Training Curves', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print(f" Training curves not found: {curves_path_stage1}")


### 7.2. Stage 2: Train SegNet với BigEarthNet

Sử dụng script `train_stage2_segnet.py` để train SegNet với BigEarthNet dataset.
**Lưu ý**: Script này sử dụng model SegNet từ `src/models/segnet.py`.


In [ ]:
# Chạy Stage 2 training sử dụng script train_stage2_segnet.py
# Script này sử dụng SegNetWithPretrainedEncoder từ src/models/segnet.py

print("="*70)
print("STAGE 2: SEGNET TRAINING WITH BIGEARTHNET")
print("="*70)
print("Sử dụng script: train_stage2_segnet.py")
print("Model: SegNetWithPretrainedEncoder (từ src/models/segnet.py)")

# Kiểm tra GPU availability
print("\n" + "-"*70)
print("GPU CHECK")
print("-"*70)

import torch
from config import DEVICE

print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

if torch.cuda.is_available():
    print(f" GPU will be used: {DEVICE}")
    print(f"   GPU count: {torch.cuda.device_count()}")
    print(f"   Current GPU: {torch.cuda.current_device()}")
    print(f"   GPU name: {torch.cuda.get_device_name(0)}")
    
    # Show GPU memory info
    if torch.cuda.is_available():
        gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
        print(f"   GPU memory: {gpu_memory:.2f} GB")
        
        # Allocate a small tensor to ensure GPU is working
        try:
            test_tensor = torch.randn(1).cuda()
            print(f"    GPU tensor allocation test: PASSED")
            del test_tensor
            torch.cuda.empty_cache()
        except Exception as e:
            print(f"    GPU tensor allocation test: FAILED - {e}")
else:
    print(f"  WARNING: CUDA not available. Training will use CPU (very slow!)")
    print(f"   Device: {DEVICE}")
    print(f"   Consider enabling GPU in Kaggle notebook settings")

print("\n" + "-"*70)
print("TRAINING PARAMETERS")
print("-"*70)
print("Các tham số có thể điều chỉnh:")
print("  --epochs: Số epochs (mặc định: 50)")
print("  --batch-size: Batch size (mặc định: 16)")
print("  --encoder-lr: Learning rate cho encoder (mặc định: 1e-5)")
print("  --decoder-lr: Learning rate cho decoder (mặc định: 1e-3)")
print("  --freeze-epochs: Số epochs freeze encoder (mặc định: 10)")
print("  --max-samples: Giới hạn số samples để train nhanh hơn (None = use all)")

# Kiểm tra encoder weights từ Stage 1
print("\n" + "-"*70)
print("ENCODER WEIGHTS CHECK")
print("-"*70)
encoder_weights_path = CHECKPOINTS_DIR / 'stage1' / 'encoder_pretrained.pth'
if encoder_weights_path.exists():
    print(f" Encoder weights từ Stage 1: {encoder_weights_path}")
    # Get file size
    import os
    size_mb = os.path.getsize(encoder_weights_path) / (1024**2)
    print(f"   Size: {size_mb:.2f} MB")
else:
    print(f"  Warning: Encoder weights không tìm thấy tại {encoder_weights_path}")
    print("   Script sẽ sử dụng ImageNet pretrained weights thay thế")

# Clear GPU cache before training
if torch.cuda.is_available():
    torch.cuda.empty_cache()
    print(f"\n GPU cache cleared")

# Chạy training với GPU
print("\n" + "="*70)
print("STARTING TRAINING (GPU will be used automatically)")
print("="*70)

# Set environment variable to ensure GPU is used (for Kaggle)
import os
os.environ['CUDA_VISIBLE_DEVICES'] = '0'  # Use first GPU

# Chạy training - DEVICE sẽ tự động được set từ config.py
!cd {project_root} && python train_stage2_segnet.py --epochs 50 --batch-size 16


In [ ]:
# Kiểm tra kết quả Stage 2
stage2_checkpoint_dir = CHECKPOINTS_DIR / 'stage2_segnet'
best_model_stage2 = stage2_checkpoint_dir / 'best_model.pth'

print("\n" + "="*70)
print("Kiểm tra kết quả Stage 2...")
print("="*70)

if best_model_stage2.exists():
    print(f" Best SegNet model saved at: {best_model_stage2}")
else:
    print(f" Best model not found at: {best_model_stage2}")

# Load và hiển thị kết quả từ log file
log_file_stage2 = project_root / 'outputs' / 'logs' / 'stage2_segnet_training.log'
if log_file_stage2.exists():
    print(f"\n✓ Training log saved at: {log_file_stage2}")
    # Đọc và hiển thị một phần log
    with open(log_file_stage2, 'r') as f:
        lines = f.readlines()
        print("\n Last few lines from training log:")
        for line in lines[-15:]:
            print(f"  {line.strip()}")
else:
    print(f"\n Training log not found at: {log_file_stage2}")


In [ ]:
# Xem thêm thông tin từ checkpoint Stage 2
if best_model_stage2.exists():
    checkpoint = torch.load(best_model_stage2, map_location='cpu')
    print("\n📊 Stage 2 SegNet Checkpoint Info:")
    if 'epoch' in checkpoint:
        print(f"  Best epoch: {checkpoint['epoch']}")
    if 'val_mean_iou' in checkpoint:
        print(f"  Best validation mIoU: {checkpoint['val_mean_iou']:.4f}")
    if 'val_pixel_accuracy' in checkpoint:
        print(f"  Best validation pixel accuracy: {checkpoint['val_pixel_accuracy']:.4f}")
    if 'val_loss' in checkpoint:
        print(f"  Best validation loss: {checkpoint['val_loss']:.4f}")
    print(f"  Checkpoint keys: {list(checkpoint.keys())}")
    
    # Verify model type
    if 'model_state_dict' in checkpoint:
        model_keys = list(checkpoint['model_state_dict'].keys())
        # Check if it's SegNet (should have decoder layers)
        has_segnet_decoder = any('decoder' in key or 'dec5' in key or 'dec4' in key for key in model_keys)
        if has_segnet_decoder:
            print("\n Model confirmed: SegNet architecture detected in checkpoint")
        else:
            print("\n Warning: SegNet decoder layers not clearly detected in checkpoint")


## 8. Visualize Results


In [ ]:
# Load và hiển thị visualizations từ Stage 2
print("\n" + "="*70)
print("Visualizing Stage 2 Results...")
print("="*70)

# Confusion matrix
cm_path_stage2 = VISUALIZATIONS_DIR / 'stage2_segnet_confusion_matrix.png'
if cm_path_stage2.exists():
    print(f"✓ Confusion matrix found: {cm_path_stage2}")
    img = plt.imread(cm_path_stage2)
    plt.figure(figsize=(12, 10))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Stage 2 SegNet: Confusion Matrix', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print(f"✗ Confusion matrix not found: {cm_path_stage2}")

# Training curves
curves_path_stage2 = VISUALIZATIONS_DIR / 'stage2_segnet_training_curves.png'
if curves_path_stage2.exists():
    print(f"✓ Training curves found: {curves_path_stage2}")
    img = plt.imread(curves_path_stage2)
    plt.figure(figsize=(18, 5))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Stage 2 SegNet: Training Curves', fontsize=14, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print(f"✗ Training curves not found: {curves_path_stage2}")


In [ ]:
# Load và hiển thị segmentation predictions
print("\n" + "="*70)
print("Visualizing Segmentation Predictions...")
print("="*70)

vis_path_stage2 = VISUALIZATIONS_DIR / 'stage2_segnet_predictions.png'
if vis_path_stage2.exists():
    print(f"✓ Predictions visualization found: {vis_path_stage2}")
    img = plt.imread(vis_path_stage2)
    plt.figure(figsize=(20, 15))
    plt.imshow(img)
    plt.axis('off')
    plt.title('Stage 2 SegNet: Predictions on Test Set', fontsize=16, fontweight='bold')
    plt.tight_layout()
    plt.show()
else:
    print(f"✗ Predictions visualization not found: {vis_path_stage2}")
    print("  Có thể script chưa chạy xong hoặc có lỗi trong quá trình training")


In [ ]:
# Load model và test predictions trực tiếp trong notebook
print("\n" + "="*70)
print("Testing Trained SegNet Model...")
print("="*70)

from config import DEVICE
from src.models.segnet import SegNetWithPretrainedEncoder
from src.data import create_bigearthnet_dataloaders, get_val_augmentation
from src.utils import SegmentationMetrics

# Load best model
if best_model_stage2.exists():
    print(f"Loading best model from: {best_model_stage2}")
    
    # Create model
    model_trained = SegNetWithPretrainedEncoder(
        encoder_name='resnet50',
        num_classes=NUM_CLASSES,
        encoder_pretrained=False
    )
    
    # Load checkpoint
    checkpoint = torch.load(best_model_stage2, map_location=DEVICE)
    if 'model_state_dict' in checkpoint:
        model_trained.load_state_dict(checkpoint['model_state_dict'])
    else:
        model_trained.load_state_dict(checkpoint)
    
    model_trained = model_trained.to(DEVICE)
    model_trained.eval()
    print("✓ Model loaded successfully!")
    
    # Test on a few samples
    val_transform = get_val_augmentation(120)
    _, val_loader_test, _ = create_bigearthnet_dataloaders(
        metadata_path=METADATA_PATH,
        data_folders=BIGEARTHNET_FOLDERS,
        reference_maps_folder=REFERENCE_MAPS_FOLDER,
        corine_to_eurosat_mapping=CORINE_TO_EUROSAT,
        train_transform=None,
        val_transform=val_transform,
        batch_size=4,
        num_workers=0,
        pin_memory=False,
        num_classes=NUM_CLASSES
    )
    
    # Get a batch
    batch = next(iter(val_loader_test))
    images = batch['image'].to(DEVICE)
    masks = batch['mask']
    
    # Predict
    with torch.no_grad():
        predictions = model_trained(images)
    
    # Visualize
    num_samples = min(4, len(images))
    fig, axes = plt.subplots(num_samples, 3, figsize=(15, 5*num_samples))
    fig.suptitle('Trained SegNet Predictions', fontsize=16, fontweight='bold')
    
    for i in range(num_samples):
        # Original image
        img_np = denormalize_image(images[i].cpu())
        axes[i, 0].imshow(img_np)
        axes[i, 0].set_title('Original Image', fontweight='bold')
        axes[i, 0].axis('off')
        
        # Ground truth mask
        mask_np = mask_to_rgb(masks[i].numpy(), COLOR_PALETTE)
        axes[i, 1].imshow(mask_np)
        axes[i, 1].set_title('Ground Truth', fontweight='bold')
        axes[i, 1].axis('off')
        
        # Prediction
        pred_mask = torch.argmax(predictions[i], dim=0).cpu().numpy()
        pred_mask_rgb = mask_to_rgb(pred_mask, COLOR_PALETTE)
        axes[i, 2].imshow(pred_mask_rgb)
        axes[i, 2].set_title('SegNet Prediction', fontweight='bold')
        axes[i, 2].axis('off')
    
    plt.tight_layout()
    plt.show()
    
    print(" Prediction visualization complete!")
else:
    print(" Model checkpoint not found. Please run Stage 2 training first.")


## 9. Summary

### Training Summary:
- **Stage 1**: Pre-trained encoder với EuroSAT classification (sử dụng `train_stage1.py`)
- **Stage 2**: Trained SegNet với BigEarthNet segmentation (sử dụng `train_stage2_segnet.py` với model từ `src/models/segnet.py`)

### Results:
- Checkpoints saved in `outputs/checkpoints/`
  - Stage 1: `outputs/checkpoints/stage1/`
  - Stage 2: `outputs/checkpoints/stage2_segnet/`
- Visualizations saved in `outputs/visualizations/`
- Logs saved in `outputs/logs/`

### Model Architecture:
- **SegNet**: Sử dụng `SegNetWithPretrainedEncoder` từ `src/models/segnet.py`
- Encoder: Pre-trained từ Stage 1 (ResNet50)
- Decoder: SegNet-style decoder với transposed convolutions
